#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces

Ultilizar api do bling para cadastro de itens   

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:
#Baixa os anexos XML dos emails

save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao

        produtos.append(produto)
    return produtos


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns).text
            
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [ ]:
produtos['Fornecedor'].unique()



In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [ ]:
for  index, produto in produtos.iterrows():
    fornecedor = produto['Fornecedor']
    codigo_produto = produto['Codigo Produto']
    descricao = produto['Descrição']
    try:
        if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
            url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
            url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
            url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
        else:
            print('Indisponivel')
            continue
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}

        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)


produtos.to_excel('produtos.xlsx', index=False)


In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright




async def run():

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))

        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()




In [ ]:
import requests
from bs4 import BeautifulSoup
import json

response = requests.get('https://msbitaqua.com.br/produto/11748/11748')
soup = BeautifulSoup(response.text, "html.parser")
script = soup.find("script", type="application/json")
data = json.loads(script.string)

produto = data['props']['pageProps']['produto']
produto

In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://msbitaqua.com.br/produto/2123/2123'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            print(data.keys())
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://www.construdigi.com.br/produto/10422/multilit-tubo-pvc-esg-75'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()


In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz

# === Configurações ===
threshold = 75  # similaridade mínima
arquivo_produtos = "produtos.xlsx"
arquivo_saida_fuzzy = "Fuzzy.xlsx"
arquivo_saida_bling = "produtos_para_bling.xlsx"

# === Leitura inicial ===
df = pd.read_excel(arquivo_produtos)

# === Agrupamento Fuzzy ===
grupos, grupo_id, grupo_coluna = {}, 1, []

for desc in df["Descrição"]:
    # Se já está em algum grupo, apenas reutiliza o ID
    gid_existente = next((gid for gid, itens in grupos.items() if desc in itens), None)
    if gid_existente:
        grupo_coluna.append(gid_existente)
        continue

    # Cria novo grupo
    grupos[grupo_id] = [desc]
    grupo_coluna.append(grupo_id)

    # Busca similares
    for match_desc, score, _ in process.extract(desc, df["Descrição"], scorer=fuzz.token_sort_ratio, limit=None):
        if score >= threshold and match_desc != desc:
            grupos[grupo_id].append(match_desc)

    grupo_id += 1

df["Grupo"] = grupo_coluna

# Filtra linhas com peso válido
df = df[df["peso"].astype(str).str.upper() != "NÃO DISPONIVEL"]

# Padroniza colunas
df.columns = df.columns.str.strip().str.lower()

# Garante nome da coluna de código
if "codigo" not in df.columns:
    for c in df.columns:
        if "código" in c or "codigo" in c:
            df.rename(columns={c: "codigo"}, inplace=True)
            break

# Cria estrutura de hierarquia
bling_data = [
    {"codigo": row["codigo produto"], "codigo pai": "" if idx == 0 else grupo.iloc[0]["codigo produto"]}
    for _, grupo in df.groupby("grupo") 
    for idx, row in grupo.reset_index(drop=True).iterrows()
]

bling_df = pd.DataFrame(bling_data)

# Junta tudo
bling_final = df.merge(bling_df, on="codigo", how="right")
bling_final.to_excel(arquivo_saida_bling, index=False)

print("✅ Planilha gerada com sucesso:", arquivo_saida_bling)



Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


SHOPE taxa de transação é sobre o total do produto então é 2% de taxa e 18 de comissão


In [ ]:
# Ler planilhas
fuzzy_df = pd.read_excel("Fuzzy.xlsx")
produtos_df = pd.read_excel("produtos.xlsx")

produtos_df = produtos_df(produtos_df["peso"].astype(str).str.upper() != "NÃO DISPONIVEL") 

# Padronizar os nomes das colunas (minúsculas e sem espaços)
fuzzy_df.columns = fuzzy_df.columns.str.strip().str.lower()
produtos_df.columns = produtos_df.columns.str.strip().str.lower()

# Verificar o nome das colunas após limpeza
print("Colunas Fuzzy:", fuzzy_df.columns.tolist())
print("Colunas Produtos:", produtos_df.columns.tolist())

# Criar lista de novos produtos
bling_data = []

# Agrupar por 'grupo'
for grupo, grupo_df in fuzzy_df.groupby("grupo"):
    grupo_df = grupo_df.reset_index(drop=True)
    codigo_principal = grupo_df.loc[0, "codigo produto"]  # o primeiro é o principal

    for idx, row in grupo_df.iterrows():
        if idx == 0:
            bling_data.append({
                "codigo": row["codigo produto"],
                "codigo pai": ""
            })
        else:
            bling_data.append({
                "codigo": row["codigo produto"],
                "codigo pai": codigo_principal
            })

# Criar DataFrame com códigos
bling_df = pd.DataFrame(bling_data)

# Corrigir nomes antes do merge (garantir existência das colunas)
if "codigo" not in produtos_df.columns:
    # tenta achar a coluna mais próxima
    similar = [c for c in produtos_df.columns if "código" in c or "codigo" in c]
    if similar:
        produtos_df.rename(columns={similar[0]: "codigo"}, inplace=True)

# Fazer o merge
bling_final = produtos_df.merge(bling_df, on="codigo", how="right")

# Exportar resultado
bling_final.to_excel("produtos_para_bling.xlsx", index=False)
print("✅ Planilha gerada: produtos_para_bling.xlsx")
